In [51]:
import numpy as np
from scipy.spatial import distance_matrix
from collections import defaultdict
import heapq
import math

data = np.array([[-82.96448724,  42.4357261 ],
               [-82.99506192,  42.41928224],
               [-82.9441534 ,  42.41618071],
               [-82.98336995,  42.39320031],
               [-82.92738974,  42.4045129 ],
               [-83.01882452,  42.43873424],
               [-83.04513325,  42.40320146],
               [-83.08105259,  42.43597913],
               [-83.1271629 ,  42.43132828],
               [-83.07867189,  42.38646699],
               [-82.97206569,  42.35611654],
               [-83.022422  ,  42.36134695],
               [-83.04922545,  42.33220139],
               [-83.06791756,  42.35533221],
               [-83.10805985,  42.3124297 ],
               [-83.07934493,  42.34494048],
               [-83.10832332,  42.36073348],
               [-83.13263769,  42.31709997],
               [-83.15606989,  42.26979623],
               [-83.13681448,  42.34585303],
               [-83.13880121,  42.40516909],
               [-83.13534311,  42.38303976],
               [-83.13933363,  42.36948879],
               [-83.173052  ,  42.37651298],
               [-83.16361542,  42.35518384],
               [-83.21046954,  42.37005731],
               [-83.20944612,  42.34331029],
               [-83.16240297,  42.42092185],
               [-83.18586971,  42.4140282 ],
               [-83.20529068,  42.40084085],
               [-83.17787423,  42.43869276],
               [-83.21718613,  42.43112063],
               [-83.24108259,  42.42199249],
               [-83.27107178,  42.42280288],
               [-83.24247996,  42.39376561],
               [-83.25319712,  42.36504472],
               [-83.23115616,  42.33742957],
               [-83.23144706,  42.36717362],
               [-83.23144716,  42.36717372],
               [-83.23144726,  42.36717382],
               [-83.23144736,  42.36717392],
               [-83.23144746,  42.36717302]])
len(data)

42

In [71]:
def find_city_center(superblocks):
    x_vals = superblocks[:,0]
    y_vals = superblocks[:,1]
    x_mean = x_vals.mean()
    y_mean = y_vals.mean()
    return np.array([x_mean, y_mean])

def id_verticies(superblocks, center, no_lines):
    # Calculate distances from central coordinate
    distances = np.linalg.norm(superblocks - center, axis=1)
    
    # Sort distances and get indices of n furthest coordinates
    furthest_indices = np.argsort(distances)[-no_lines*2:]
    
    # Extract the furthest coordinates
    furthest_coords = superblocks[furthest_indices]
    
    return furthest_coords

def assign_verticies(verticies):
    all_pairs = list(combinations(verticies, 2))

    # Sort pairs by distance
    sorted_pairs = sorted(all_pairs, key=lambda pair: np.linalg.norm(pair[0] - pair[1]), reverse=True)
    
    # Group the coordinates into pairs
    furthest_pairs = []
    paired_indices = set()
    for pair in sorted_pairs:
        idx1 = np.where((verticies == pair[0]).all(axis=1))[0][0]
        idx2 = np.where((verticies == pair[1]).all(axis=1))[0][0]
        if idx1 not in paired_indices and idx2 not in paired_indices:
            furthest_pairs.append(pair)
            paired_indices.add(idx1)
            paired_indices.add(idx2)
    
    return furthest_pairs

#COMBINATIONS TAKEN FROM PYTHON DOCUMENTATION
def combinations(iterable, r):
    # combinations('ABCD', 2) → AB AC AD BC BD CD
    # combinations(range(4), 3) → 012 013 023 123
    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    yield tuple(pool[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i+1, r):
            indices[j] = indices[j-1] + 1
        yield tuple(pool[i] for i in indices)

#Calculates the length of one line. Helper method to get_lowest_mean. 
def pythagerize(cords):
    total_len = 0
    for i in range(0, len(cords)-1, 1):
        x1, y1 = cords[i]
        x2, y2 = cords[i + 1]
        app_len = math.sqrt((x2-x1)**2 + (y2-y1)**2)
        total_len += app_len
    return total_len
    
def get_lowest_mean(combos, term):
    #concatenate the starting and terminal node onto combinations
    route_len = float('inf')
    best_route = []
    for i in range(0, len(combos), 1):
        #unpack from tuple
        inner_pts = list(combos[i])
        test_pts = [term[0]]
        for j in inner_pts:
            test_pts.append(j)
        test_pts.append(term[1])
        test_len = pythagerize(test_pts)
        if (test_len < route_len):
            route_len = test_len
            best_route = test_pts
    return route_len, best_route

def remove_pts(data, to_remove):
    for i in to_remove:
        if (i in data):
            data.remove(i)
    return data

#Creates best train system given parameters.
def create_lines(data_lt, tup):
    best_system = []
    system_len = 0
    n=0
    while(len(data_lt) > 0):
        combos = list(combinations(data_lt, 5))
        terminicies = (tup[n], tup[n+1])
        add_len, add_route = get_lowest_mean(combos, terminicies)
        best_system.append(add_route)
        system_len += add_len
        data_lt = remove_pts(data_lt, add_route)
        n+=2
    return system_len, best_system

def compare_lines(best_len, new_len, best_system, new_system):
    #Returns whichever length/system is shorter.
    pass

In [70]:
no_trains = 6
cen = find_city_center(data)
cords = id_verticies(data, cen, no_trains)
pairs = assign_verticies(cords)

#Extracts terminals from 2D array and puts them into a 1D list.
extract = []
for i in pairs:
    extract.append(i[0])
    extract.append(i[1])

#Changes data from a np.array() to a list.
data_list = []
for i in range(0, len(data), 1):
    data_list.append(data[i,:])

#Converts data into tuple dtype.
tup = [tuple(arr) for arr in extract]
data_lt = [tuple(arr) for arr in data_list]

#Removes terminals from the inner points.
data_lt = remove_pts(data_lt, tup)


create_lines(data_lt, tup)



(2.7291466156912647,
 [[(-83.27107178, 42.42280288),
   (-83.23144706, 42.36717362),
   (-83.23144716, 42.36717372),
   (-83.23144726, 42.36717382),
   (-83.23144736, 42.36717392),
   (-83.23144746, 42.36717302),
   (-82.92738974, 42.4045129)],
  [(-83.25319712, 42.36504472),
   (-83.21046954, 42.37005731),
   (-83.16240297, 42.42092185),
   (-83.18586971, 42.4140282),
   (-83.20529068, 42.40084085),
   (-83.17787423, 42.43869276),
   (-82.9441534, 42.41618071)],
  [(-83.24247996, 42.39376561),
   (-83.13880121, 42.40516909),
   (-83.13534311, 42.38303976),
   (-83.13933363, 42.36948879),
   (-83.173052, 42.37651298),
   (-83.16361542, 42.35518384),
   (-82.96448724, 42.4357261)],
  [(-83.24108259, 42.42199249),
   (-83.1271629, 42.43132828),
   (-83.07867189, 42.38646699),
   (-83.04922545, 42.33220139),
   (-83.06791756, 42.35533221),
   (-83.07934493, 42.34494048),
   (-82.97206569, 42.35611654)],
  [(-83.15606989, 42.26979623),
   (-83.10805985, 42.3124297),
   (-83.13263769, 42.31